In [ ]:
import os
import sys

print("Current working directory:", os.getcwd())

# Add the project root directory to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)
print("Added to sys.path:", project_root)

# Now try to import the modules
try:
    from models.state import SharedState
    from config.config import Config
    from agents.url_generation_agent import url_generation_agent
    from agents.article_extraction_agent import article_extraction_agent
    from tools.searching.google_cse import GoogleCSE
    from agents.router_agent import router_agent
    print("All modules imported successfully")
except ImportError as e:
    print(f"Failed to import: {e}")

# Print the contents of the project root directory
print("Contents of project root:", os.listdir(project_root))

Unit Test for GoogleCSE Class
Test Objective:
- Verify that the GoogleCSE class correctly handles successful API calls and error scenarios.
Expected Results:
- Successful search returns a list of URLs.
- API errors are handled gracefully, returning an empty list.
Variables Used:
- Mocked API responses for success and error cases.

In [ ]:
class TestGoogleCSE(unittest.IsolatedAsyncioTestCase):

    async def test_successful_search(self):
        async def mock_get(*args, **kwargs):
            class MockResponse:
                status = 200
                async def json(self):
                    return {
                        "items": [
                            {"link": "http://example.com/1"},
                            {"link": "http://example.com/2"},
                        ]
                    }
            return MockResponse()
        
        with patch('aiohttp.ClientSession.get', new=mock_get):
            cse = GoogleCSE(api_key='test_key', cx='test_cx')
            results = await cse.search('test query')
            self.assertEqual(results, ['http://example.com/1', 'http://example.com/2'])

    async def test_api_error(self):
        async def mock_get(*args, **kwargs):
            class MockResponse:
                status = 403
                async def text(self):
                    return "Forbidden"
            return MockResponse()
        
        with patch('aiohttp.ClientSession.get', new=mock_get):
            cse = GoogleCSE(api_key='test_key', cx='test_cx')
            results = await cse.search('test query')
            self.assertEqual(results, [])

    async def test_timeout(self):
        with patch('aiohttp.ClientSession.get', new_callable=AsyncMock) as mock_get:
            mock_get.side_effect = asyncio.TimeoutError
            cse = GoogleCSE(api_key='test_key', cx='test_cx')
            results = await cse.search('test query')
            self.assertEqual(results, [])

# Run TestGoogleCSE
async def run_google_cse_tests():
    suite = unittest.TestLoader().loadTestsFromTestCase(TestGoogleCSE)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

asyncio.run(run_google_cse_tests())

Unit Test for url_generation_agent
Test Objective:
- Verify that the agent correctly generates search terms and decides on the search agent.
Expected Results:
- Correct search terms are generated.
- Agent selection logic works as expected.
Variables Used:
- Mocked OpenAI API responses.

In [ ]:
class TestURLGenerationAgent(unittest.IsolatedAsyncioTestCase):

    async def test_url_generation(self):
        state = SharedState()
        state.user_query = "Test query"
        state.config = Config()
        # Mock OpenAI responses
        with patch('openai.ChatCompletion.acreate', new_callable=AsyncMock) as mock_openai:
            mock_openai.return_value = AsyncMock(choices=[AsyncMock(message={'content': 'search term1, search term2'})])
            await url_generation_agent(state)
            self.assertIn('search term1', state.search_terms)
            self.assertIn('search term2', state.search_terms)

# Run TestURLGenerationAgent
async def run_url_generation_test():
    test = TestURLGenerationAgent()
    await test.test_url_generation()

asyncio.run(run_url_generation_test())

Unit Test for article_extraction_agent
Test Objective:
- Verify that the agent correctly extracts data from articles.
Expected Results:
- Extracted data is stored in the state.
- Errors are handled gracefully.
Variables Used:
- Mocked OpenAI API responses.

In [ ]:
class TestArticleExtractionAgent(unittest.IsolatedAsyncioTestCase):

    async def test_article_extraction(self):
        state = SharedState()
        state.articles = {'http://example.com': 'Article content'}
        state.config = Config()
        # Mock OpenAI responses
        with patch('openai.ChatCompletion.acreate', new_callable=AsyncMock) as mock_openai:
            mock_openai.return_value = AsyncMock(choices=[AsyncMock(message={'content': '{"key": "value"}'})])
            await article_extraction_agent(state)
            self.assertIn('http://example.com', state.extracted_data)
            self.assertEqual(state.extracted_data['http://example.com'], {'key': 'value'})

# Run TestArticleExtractionAgent
async def run_article_extraction_test():
    test = TestArticleExtractionAgent()
    await test.test_article_extraction()

asyncio.run(run_article_extraction_test())

Integration Test for the Entire Workflow
Test Objective:
- Verify that the agents work together to complete the workflow.
Expected Results:
- Workflow completes successfully.
- All state variables are appropriately populated.
Variables Used:
- Mocked API responses for OpenAI, Google CSE, Tavily, and Jina.

In [ ]:
class TestWorkflow(unittest.IsolatedAsyncioTestCase):

    async def test_full_workflow(self):
        state = SharedState()
        state.user_query = "Test query"
        state.config = Config()

        # Mock OpenAI API
        with patch('openai.ChatCompletion.acreate', new_callable=AsyncMock) as mock_openai, \
             patch('tools.searching.google_cse.GoogleCSE.search', new_callable=AsyncMock) as mock_google_search, \
             patch('tools.scraping.jina_scraper.JinaScraper.scrape', new_callable=AsyncMock) as mock_jina_scrape:
            
            # Mock responses
            mock_openai.return_value = AsyncMock(choices=[AsyncMock(message={'content': 'search term1, search term2'})])
            mock_google_search.return_value = ['http://example.com']
            mock_jina_scrape.return_value = 'Article content'

            # Run the workflow
            await router_agent(state)
            self.assertEqual(state.next_step, 'scraper_selection')
            await router_agent(state)
            self.assertEqual(state.next_step, 'article_extraction')
            await router_agent(state)
            self.assertEqual(state.next_step, 'review')
            await router_agent(state)
            self.assertEqual(state.next_step, 'knowledge_graph_upload')
            await router_agent(state)
            self.assertEqual(state.next_step, 'end')

# Run TestWorkflow
async def run_workflow_test():
    test = TestWorkflow()
    await test.test_full_workflow()

asyncio.run(run_workflow_test())